In [1]:
import mne
from scipy import signal
from scipy.fft import fftshift
import matplotlib.pyplot as plt
import numpy as np
import os
import math

In [2]:
def fileToLabeled(path= "./edfs/S005/S005E01.edf"):
  data = mne.io.read_raw_edf(path,verbose='CRITICAL')
  filtered=data.load_data(verbose='CRITICAL').filter(0.5, 43, verbose='CRITICAL')
  raw_data=filtered.get_data() 
  lst=[]
  for i in range(len(data.ch_names)):
    if i==8: continue
    start_range=0
    sec_range=180
    channel=i
    f, t, specs = signal.spectrogram(raw_data[channel][256*start_range:256*sec_range],256, nperseg=256,noverlap=0)
    theta_new=[]
    for a in range(sec_range-start_range):
      theta_new.append((specs[4][a]+specs[5][a]+specs[6][a]+specs[7][a]))

    beta_new=[]
    for a in range(sec_range-start_range):
      beta_new.append((specs[13][a]+specs[14][a]+specs[15][a]+specs[16][a]+specs[17][a]+specs[18][a]+specs[19][a]+specs[20][a]
              +specs[21][a]+specs[22][a]+specs[23][a]+specs[24][a]+specs[25][a]))

    ratio=[theta_new[i]/beta_new[i] for i in range(sec_range-start_range)]
    lst.append(ratio)
  lst.append([(0 if (i<60 or i>120) else 1) for i in t])
  res=np.array(lst).T
  return res

def folderToLabeled(path="./edfs/S001/"):
  fnames = os.listdir(path)
  lst=[]
  for i in fnames:
    #print(f"!!!!!!!!!!!! Processing {i}...")
    rat=fileToLabeled(path+i)
    for j in rat:
      lst.append(j)
  return np.array(lst)

def projectTo(path="./edfs/"):
  pass


In [7]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from statistics import stdev
from tqdm import tqdm
def getData(data :np.ndarray, test=0.2, shuffled=True, trim=0.5):
    copied=data.copy()
    if shuffled:
        np.random.shuffle(copied)
    cnt=len(copied)
    testr=math.floor(cnt*test)
    trainr=math.floor(cnt*(1-test))
    return (copied[:trainr], copied[trainr:trainr+testr])
def classify(data):
    new=data.T
    input=new[:13]
    label=new[13]
    return (input.T, label)

def work():
    res=[]
    ldaarr=[]
    for i in tqdm([f"S{i:03}" for i in range(1,31)]):
        data=folderToLabeled(f"./edfs/{i}/")
        (train, test)=getData(data)
        x, y=classify(train)
        x_test, y_test=classify(test)
        lda=LinearDiscriminantAnalysis()
        scaled = x#StandardScaler().fit_transform(x.T)
        scaled_test = x_test#StandardScaler().fit_transform(x_test.T)
        lda.fit(scaled, y)
        y_predict=lda.predict(scaled_test)
        y_train_pred=lda.predict(scaled)
        res.append((i, metrics.accuracy_score(y, y_train_pred), metrics.accuracy_score(y_test, y_predict)))
        ldaarr.append(lda)
    sum=0
    stdarr=[]
    bestind=0
    bestacc=0
    besta=""
    for ind, i in enumerate(res):
        print(f"{i[0]} -> train:{i[1]}, test:{i[2]}")
        sum+=i[2]
        stdarr.append(i[2])
        if i[2]>bestacc:
            bestacc=i[2]
            bestind=ind
            besta=i[0]
    print(f"avg:{sum/len(res)}, std:{stdev(stdarr)}")
    print(f"bestacc:{bestacc}, besta:{besta}, ind:{ind}")
    return (ldaarr, bestind, besta)
(ldas, ind, bestname)=work()



100%|██████████| 30/30 [00:06<00:00,  4.47it/s]

S001 -> train:0.6736111111111112, test:0.6875
S002 -> train:0.6614583333333334, test:0.6875
S003 -> train:0.6805555555555556, test:0.6597222222222222
S004 -> train:0.6875, test:0.6458333333333334
S005 -> train:0.6979166666666666, test:0.6597222222222222
S006 -> train:0.7065972222222222, test:0.7152777777777778
S007 -> train:0.671875, test:0.6527777777777778
S008 -> train:0.7083333333333334, test:0.7222222222222222
S009 -> train:0.6875, test:0.6388888888888888
S010 -> train:0.6510416666666666, test:0.6944444444444444
S011 -> train:0.6440972222222222, test:0.7361111111111112
S012 -> train:0.6614583333333334, test:0.6875
S013 -> train:0.7013888888888888, test:0.5625
S014 -> train:0.6753472222222222, test:0.6666666666666666
S015 -> train:0.6545138888888888, test:0.7291666666666666
S016 -> train:0.6788194444444444, test:0.6944444444444444
S017 -> train:0.6701388888888888, test:0.6875
S018 -> train:0.6736111111111112, test:0.7222222222222222
S019 -> train:0.6736111111111112, test:0.666666666

In [9]:
import pandas as pd
from ntpath import basename
def ExportToCsv(model :LinearDiscriminantAnalysis, path="./edfs/S012/S012E03.edf"):
    def softmax(x):
        f_x = np.exp(x) / np.sum(np.exp(x))
        return f_x
    print(f"Working on {path}")
    ratio=fileToLabeled(path)
    (input, label)=classify(ratio)
    predicted=model.predict(input)
    score=metrics.accuracy_score(label, predicted)
    print(f"csv accuracy:{score}")
    #csv
    work=np.array(input)
    for i, d in enumerate(input):
        work[i]=softmax(d)*(0.5 if predicted[i] is 0 else 1)
    df = pd.DataFrame(work)
    root, extension = os.path.splitext(path)
    df.to_csv(basename(root)+".csv", index=False)
ExportToCsv(ldas[ind], f"./edfs/{bestname}/{bestname}E03.edf")

Working on ./edfs/S011/S011E03.edf
csv accuracy:0.6555555555555556
